In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import os
tf.random.set_seed(101)


In [ ]:
df = pd.read_csv('/kaggle/input/cryptocurrencypricehistory/coin_Bitcoin.csv')
df = df.dropna()
df = df.sort_values('Date').reset_index(drop=True)

# Representation of collected Ehterium data 

In [ ]:
df.head()

# Data preprocessing part

In [ ]:
df = df.drop(['Open', 'High', 'Low', 'Volume', 'Marketcap','Volume', 'Symbol', 'Name', 'SNo'], axis = 1)
df['Close'] = df['Close'].astype(float)
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
last_date = df['Date'].iloc[-1]
df['Date'] = df['Date'].apply(datetime.timestamp)

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
df_scaled = sc.fit_transform(df)

# Generating of sequences with certain period of days

In [ ]:
window = 60
X = []
y = []
for i in range(window, len(df_scaled)):
    X.append(np.reshape(df_scaled[i-window:i,0], (window, 1)))
    y.append(df_scaled[i,1])

X = np.stack(X)
y = np.stack(y)

In [ ]:
X.shape

# **Splitting data to train test parts**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle = False)

# LSTM+GRU model architecture

In [ ]:
model = models.Sequential()
model.add(LSTM(units = 100, use_bias=True, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 1)))
# model.add(Dropout(0.2))
model.add(GRU(units = 50, activation = 'relu', return_sequences = True, use_bias=True))
# model.add(Dropout(0.2))
model.add(LSTM(units = 10, use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(units = 1))


In [ ]:
model.compile(optimizer = 'Adamax', loss = 'mse', metrics = 'mae')
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose=1)

In [ ]:
plt.rcParams["figure.figsize"] = (15,8) # plotsize
plt.plot(history.history['mae'], color = '#F77800', linewidth=2) #plothistory
plt.grid(color='#ADADAD', linestyle='--', linewidth=1) # setup grid
plt.xlabel('Epoch', fontsize = 20) # x label
plt.ylabel('Mean absolute error', fontsize = 20) # y label

# Forecasting on existing dataset

In [ ]:
Train_predicted_values = model.predict(X_train)
Train_predicted = [[],[]]

for i in range(len(X_train)):
    Train_predicted[0].append(df_scaled[i+window, 0])
    Train_predicted[1].append(Train_predicted_values[i])

Train_predicted = np.array(Train_predicted).T
Train_inversed_values = sc.inverse_transform(Train_predicted)
Train_inversed_values = pd.DataFrame(Train_inversed_values)oculus 
Train_inversed_values[0] = Train_inversed_values[0].apply(lambda x: datetime.fromtimestamp(x))


Test_predicted_values = model.predict(X_test)
Test_predicted = [[],[]]

for i in range(len(X_test)):
    Test_predicted[0].append(df_scaled[i+window+len(X_train), 0])
    Test_predicted[1].append(Test_predicted_values[i])

Test_predicted = np.array(Test_predicted).T
Test_inversed_values = sc.inverse_transform(Test_predicted)
Test_inversed_values = pd.DataFrame(Test_inversed_values)
Test_inversed_values[0] = Test_inversed_values[0].apply(lambda x: datetime.fromtimestamp(x))

MSE = mean_squared_error(Test_inversed_values[1], y_test)
MAE = mean_absolute_error(Test_inversed_values[1], y_test)
print(f"MSE:{MSE}\nMAE:{MAE}")

In [ ]:
plt.rcParams["figure.figsize"] = (22,10) # plotsize
plt.rcParams['axes.facecolor'] = '#ffffff'
plt.grid(color='#ADADAD', linestyle='-.', linewidth=1)
plt.plot(df['Date'].apply(datetime.fromtimestamp), df['Close'], color = '#0DD527', linewidth=2, label = "ETH data")
plt.plot(Train_inversed_values[0],Train_inversed_values[1], color = '#F77800', linewidth=2, label = "Train data")
plt.plot(Test_inversed_values[0],Test_inversed_values[1], color = '#0F97FF', linewidth=2, label = "Test data")
plt.legend(fontsize='x-large')
plt.show()

# N days forecast

In [ ]:
days = 60
end_date = last_date + timedelta(days)
time_range = pd.date_range(start=last_date,end=end_date)
timestamp_time_range = time_range.to_series().apply(datetime.timestamp).values
vals = [0]*len(time_range)
data_for_scaling = np.array([timestamp_time_range, vals]).T
scaled_forecast = sc.transform(data_for_scaling)
scaled_forecast = np.concatenate((df_scaled, scaled_forecast), axis=0)

X_forecast_part = []
for i in range(len(df_scaled)-window, len(scaled_forecast)):
    X = np.reshape(scaled_forecast[i-window:i,0], (window, 1))
    X_forecast_part.append(X)

X_forecast_part = np.stack(X_forecast_part)oculus 

forecast = model.predict(X_forecast_part)
forecast_arr = [[], []]

for j in range(window, len(forecast)):
    forecast_arr[0].append(scaled_forecast[j, 0])
    forecast_arr[1].append(forecast[j-window])

forecast_predicted_arr = np.array(forecast_arr, dtype=np.float64).T
forecast_inversed_values = sc.inverse_transform(forecast_predicted_arr)
forecast = pd.DataFrame(forecast_inversed_values)


plt.grid(color='#ADADAD', linestyle='-.', linewidth=1)
plt.plot(df['Date'].apply(datetime.fromtimestamp), df['Close'], color = '#0DD527', linewidth=2, label = "ETH data")
plt.plot(Train_inversed_values[0],Train_inversed_values[1], color = '#F77800', linewidth=2, label = "Train data")
plt.plot(Test_inversed_values[0],Test_inversed_values[1], color = '#0F97FF', linewidth=2, label = "Test data")
plt.plot(time_range,forecast[1], color = '#00FFA9', linewidth=2, label = "forecast")
plt.legend(fontsize='x-large')
plt.show()